# Fluid-structure interaction

Michael Neunteufel

In this section we show how to solve a fluid-structure multi-physics problem. First, we will describe the involved equations, Arbitrary Lagrangian Eulerian (ALE) methods, interface conditions, and the problem of deformation extension. Then, two types of discretizing the fluid part are presented
* [Taylor-Hood elements](fsi_TaylorHood.ipynb)
* [H(div)-conforming HDG](fsi_HDiv.ipynb) 

__Equations on fluid and solid__

The Navier-Stokes equations with symmetric stress, velocity $u$, and pressure $p$ are considered for the fluid part

\begin{align*}
&\rho\frac{\partial u}{\partial t} + \rho(u\cdot \nabla)u -\rho\nu \mathrm{div}(\nabla u^\top+\nabla u) + \nabla p = f,\\
&\mathrm{div}(u) = 0,
\end{align*}

where $\rho$ and $\nu$ are the density and viscosity, respectively. For the solid we use the elastic wave equation with displacement $d$ and $P=F\Sigma$ the first Piola Kirchhoff stress tensor

\begin{align*}
\rho\frac{\partial^2 d}{\partial t^2} - \mathrm{div}(F\Sigma) = g
\end{align*}

where $\rho$ is the density. We consider a St.Venant-Kirchhoff material law

\begin{align*}
\Sigma = 2\mu E + \lambda \mathrm{tr}(E)I,\quad \text{ where } F = I  + \nabla d,\quad E=\frac{1}{2}(F^\top F-I)
\end{align*}

with $\mu$, $\lambda$ the Lame parameters.

__Navier Stokes in ALE (Arbitrary Lagrangian Eulerian) formulation__

The Navier-Stokes equations are given in Eulerian form, whereas the elasticity problem is formulated in Lagrangian coordinates. Thus, to couple these two equations we use the Arbitrary Lagrangian Eulerian form (short ALE) for the fluid part, see e.g. [<a href="https://doi.org/10.1002/0470091355.ecm009">Donea, Huerta, Ponthot, Rodriguez. Arbitrary Lagrangian-Eulerian Methods. <i> In: Encyclopedia of Computational Mechanics</i>, 2004</a>].

We will always work on a fixed reference domain. Therefore, we will transform the Navier-Stokes equations back to it. Let $\Phi(x,t)=\mathrm{id} + d$
describe the movement of the mesh, where $d$ is the displacement. Then we can relate the velocity on the deformed configuration $u$ to the fixed configuration $\hat{u}$ by e.g.

\begin{align*}
u\circ\Phi=\hat{u}
\end{align*}

and use the chain rule

\begin{align*}
\nabla_xu\circ\Phi = \nabla_{\hat{x}}\hat{u}F^{-1},
\end{align*}

where $F=\nabla\Phi$ denotes the deformation gradient. Next, we take the time derivative and get again with the chain rule

\begin{align*}
\frac{\partial u}{\partial t}\circ\Phi = \frac{\partial\hat{u}}{\partial t}-\nabla_{\hat{x}}F^{-1}\frac{\partial \Phi}{\partial t} = \frac{\partial\hat{u}}{\partial t}-\nabla_{\hat{x}}F^{-1}\dot{d}.
\end{align*}

The time derivative of the deformation $\dot{d}$
is called the mesh velocity and describes the relative movement of the mesh. Next we apply the transformation theorem, from where we get the determinant of the deformation gradient $J=\det(F)$. With this the ALE Navier-Stokes equations read in weak form

\begin{align*}
&\int_{\Omega^f}J(\rho\frac{\partial \hat{u}}{\partial t}\cdot\hat{v}+\rho((\hat{u}-\dot{d})\cdot\nabla)\hat{u}F^{-1}\cdot\hat{v}+2\rho\nu \mathrm{sym}(\nabla\hat{u}F^{-1}):\mathrm{sym}(\nabla\hat{v}F^{-1})-\mathrm{tr}(\nabla\hat{v}F^{-1})p)\,dx = 0&&\qquad \forall \hat{v},\\
&\int_{\Omega^f}J\,\mathrm{tr}(\nabla\hat{u}F^{-1})q\,dx = 0&&\qquad \forall q.
\end{align*}

__Elastic wave equation as first order system__

The elastic wave equation does not have to be transformed as it is already stated in Lagrangian form. We rewrite it as a system of first order equations in time by introducing the solid velocity $u=\dot{d}$. The variational formulation is therefore given by

\begin{align*}
& \int_{\Omega^s}\frac{\partial d}{\partial t}\cdot v \,dx = \int_{\Omega^s}u\cdot v\,dx &&\qquad \forall v,\\
&\int_{\Omega^s}\rho\frac{\partial u}{\partial t}\cdot w+(F\Sigma):\nabla w\,dx = 0&&\qquad \forall w.
\end{align*}

__Interface conditions__
The interface conditions for fluid-structure interaction have also to be considered before we can couple the equations. The velocity and displacement have to be continuous over the interface and the forces from the fluid and the solid have to be in equilibrium

\begin{align*}
\int_{\Gamma_I}\sigma^sn\,ds = \int_{\Gamma_I}\sigma^fn\,ds,
\end{align*}

where $\sigma^s$ and $\sigma^f$ denote the stresses from the solid and fluid part, respectively.

As we are using a monolithic approach, where both equations are solved together in each time step, this equation is always fulfilled as a natural condition in weak sense as long as the deformation and velocity field is continuous over the interface. Therefore, we add both, the Navier-Stokes and the elastic wave equation, and then simply neglect the two force terms.

__Deformation extension into fluid domain__

The last important ingredient is the deformation extension. The mesh velocity $\dot{d}$
in the Navier-Stokes equations is artificial and has to be constructed from the given displacement of the solid on the interface. A very simple approach would be to solve a Poisson problem on the fluid domain with the solid displacement as dirichlet boundary conditions on the interface and homogeneous dirichlet conditions on the other boundaries. This works only for small deformations as the triangles might get pressed through others leading to a break-down. Thus, we will penalize the volume compression on the one hand and play around with a space dependent function to manipulate the stiffness of the problem, which yields a nonlinear elasticity problem with the material law of Neo-Hook

\begin{align*}
N = h(x)\mu(\mathrm{tr}(E)+\frac{2\mu}{\lambda}\det(C)^{-\frac{\lambda}{2\mu}}-1),
\end{align*}

where $h(x)$
denotes the space dependent function.  The extension can have a negative effect on the elastic wave equations on the solid domain. Therefore, we multiply it with a small parameter to minimize this effect.